# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [18]:
ls

 Volume in drive C is OS
 Volume Serial Number is A28A-C3E7

 Directory of c:\Users\mkiganane\Documents\moringa_school\Phase_2\data_admin_101\dsc-database-admin-101-lab

24/02/2025  08:15    <DIR>          .
24/02/2025  08:15    <DIR>          ..
24/02/2025  07:08    <DIR>          .github
24/02/2025  07:08                69 .gitignore
24/02/2025  07:08                95 .learn
24/02/2025  07:08             1,213 contact_list.pickle
24/02/2025  07:08             1,849 CONTRIBUTING.md
24/02/2025  07:08    <DIR>          env
24/02/2025  08:21            10,808 index.ipynb
24/02/2025  07:08             1,371 LICENSE.md
24/02/2025  07:08             3,705 README.md
24/02/2025  08:15            12,288 school.sqlite.db
               8 File(s)         31,398 bytes
               4 Dir(s)  339,991,691,264 bytes free


In [19]:
# Import necessary packages
import pandas as pd
import sqlite3

In [20]:
# Create the database school.sqlite 
conn = sqlite3.connect("school.sqlite.db")
cur = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [21]:
# Your code here
cur.execute('''
    CREATE TABLE IF NOT EXISTS contact_info (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        role TEXT CHECK(role IN ('student', 'staff')) NOT NULL,
        telephone_number TEXT NOT NULL,
        street TEXT NOT NULL,
        city TEXT NOT NULL,
        state TEXT NOT NULL,
        zipcode TEXT NOT NULL
    )
''')

# Commit the changes to the database
conn.commit()

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [22]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [23]:
print(contacts[:3])  # Print the first 3 entries to check structure

[{'firstName': 'Christine', 'lastName': 'Holden', 'role': 'staff', 'telephone ': 2035687697, 'street': '1672 Whitman Court', 'city': 'Stamford', 'state': 'CT', 'zipcode ': '06995'}, {'firstName': 'Christopher', 'lastName': 'Warren', 'role': 'student', 'telephone ': 2175150957, 'street': '1935 University Hill Road', 'city': 'Champaign', 'state': 'IL', 'zipcode ': '61938'}, {'firstName': 'Linda', 'lastName': 'Jacobson', 'role': 'staff', 'telephone ': 4049446441, 'street': '479 Musgrave Street', 'city': 'Atlanta', 'state': 'GA', 'zipcode ': '30303'}]


In [37]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    cur.execute('''
        INSERT INTO contact_info (first_name, last_name, role, telephone_number, street, city, state, zipcode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        contact['firstName'],  # Fix capitalization
        contact['lastName'],   # Fix capitalization
        contact['role'],
        str(contact['telephone ']).strip(),  # Fix key name & ensure string format
        contact['street'],
        contact['city'],
        contact['state'],
        contact['zipcode '].strip()  # Fix key name & strip whitespace
    ))

# Commit changes to save the data
conn.commit()

**Query the Table to Ensure it is populated**

In [38]:
# Your code here 
# Fetch and display the first 10 rows from contact_info
cur.execute("SELECT * FROM contact_info LIMIT 10")
rows = cur.fetchall()

# Print the retrieved rows
for row in rows:
    print(row)


(None, 'Christine', 'Holden', 'staff', 2035687697, '1672 Whitman Court', 'Stamford', 'CT', 6995)
(None, 'Christopher', 'Warren', 'student', 2175150957, '1935 University Hill Road', 'Champaign', 'IL', 61938)
(None, 'Linda', 'Jacobson', 'staff', 4049446441, '479 Musgrave Street', 'Atlanta', 'GA', 30303)
(None, 'Andrew', 'Stepp', 'student', 7866419252, '2981 Lamberts Branch Road', 'Hialeah', 'Fl', 33012)
(None, 'Jane', 'Evans', 'student', 3259909290, '1461 Briarhill Lane', 'Abilene', 'TX', 79602)
(None, 'Jane', 'Evans', 'student', 3259909290, '1461 Briarhill Lane', 'Abilene', 'TX', 79602)
(None, 'Mary', 'Raines', 'student', 9075772295, '3975 Jerry Toth Drive', 'Ninilchik', 'AK', 99639)
(None, 'Ed', 'Lyman', 'student', 5179695576, '3478 Be Sreet', 'Lansing', 'MI', 48933)
(None, 'Christine', 'Holden', 'staff', 2035687697, '1672 Whitman Court', 'Stamford', 'CT', 6995)
(None, 'Christopher', 'Warren', 'student', 2175150957, '1935 University Hill Road', 'Champaign', 'IL', 61938)


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [26]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [34]:
# Create the grades table
cur.execute('''
    CREATE TABLE IF NOT EXISTS grades (
        userId INTEGER NOT NULL,
        courseId INTEGER NOT NULL,
        grade TEXT NOT NULL,
        PRIMARY KEY (userId, courseId)
    )
''')

# Commit the changes to the database
conn.commit()


## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [40]:
# Find the duplicate entry
cur.execute('''
    SELECT first_name, last_name, role, telephone_number, street, city, state, zipcode, COUNT(*)
    FROM contact_info
    GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
    HAVING COUNT(*) > 1
''')

duplicates = cur.fetchall()

for row in duplicates:
    print(row)  # Print duplicate entries


('Andrew', 'Stepp', 'student', 7866419252, '2981 Lamberts Branch Road', 'Hialeah', 'Fl', 33012, 4)
('Christine', 'Holden', 'staff', 2035687697, '1672 Whitman Court', 'Stamford', 'CT', 6995, 4)
('Christopher', 'Warren', 'student', 2175150957, '1935 University Hill Road', 'Champaign', 'IL', 61938, 4)
('Ed', 'Lyman', 'student', 5179695576, '3478 Be Sreet', 'Lansing', 'MI', 48933, 4)
('Jane', 'Evans', 'student', 3259909290, '1461 Briarhill Lane', 'Abilene', 'TX', 79602, 8)
('Linda', 'Jacobson', 'staff', 4049446441, '479 Musgrave Street', 'Atlanta', 'GA', 30303, 4)
('Mary', 'Raines', 'student', 9075772295, '3975 Jerry Toth Drive', 'Ninilchik', 'AK', 99639, 4)


In [41]:
# Delete the duplicate entry
cur.execute('''
    DELETE FROM contact_info
    WHERE ROWID NOT IN (
        SELECT MIN(ROWID)
        FROM contact_info
        GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
    )
''')

conn.commit()

In [43]:
# Check that the duplicate entry was removed
cur.execute('''
    SELECT first_name, last_name, role, telephone_number, street, city, state, zipcode, COUNT(*)
    FROM contact_info
    GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
    HAVING COUNT(*) > 1
''')

print(cur.fetchall())  


[]


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [44]:
# Update Ed's address
cur.execute('''
    UPDATE contact_info
    SET street = ?, city = ?, state = ?, zipcode = ?
    WHERE first_name = ? AND last_name = ?
''', ('2910 Simpson Avenue', 'York', 'PA', '17403', 'Ed', 'Lyman'))

conn.commit()

In [45]:
# Query the database to ensure the change was made
cur.execute('''
    SELECT * FROM contact_info
    WHERE first_name = 'Ed' AND last_name = 'Lyman'
''')

print(cur.fetchall())  

[(None, 'Ed', 'Lyman', 'student', 5179695576, '2910 Simpson Avenue', 'York', 'PA', 17403)]


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [46]:
# Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 